In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib

#AMAZON_URL
amazon_URL = 'https://www.amazon.de/dp/B0BDJMH6JP/ref=redir_mobile_desktop?_encoding=UTF8&aaxitk=f581a3969bfbfebe32df4c6933031cd4&content-id=amzn1.sym.6f8b36f0-c2c9-44f2-97a8-5b151d2fc9c7%3Aamzn1.sym.6f8b36f0-c2c9-44f2-97a8-5b151d2fc9c7&hsa_cr_id=0&pd_rd_plhdr=t&pd_rd_r=d6cd7d9c-b045-4275-bb01-47e87d5bf63a&pd_rd_w=enw7T&pd_rd_wg=XL5JS&qid=1686683338&ref_=sbx_be_s_sparkle_mcd_asin_1_img&sr=1-2-e0fa1fdd-d857-4087-adda-5bd576b25987'

#Saturn_URL
saturn_URL='https://www.saturn.de/de/product/_apple-iphone-14-pro-max-256-gb-space-schwarz-dual-sim-2832775.html'

#Mediamarkt_URL
mediamarkt_URL='https://www.mediamarkt.de/de/product/_apple-iphone-14-pro-max-256-gb-space-schwarz-dual-sim-2832775.html'


headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}


# Send requests and create BeautifulSoup objects for each website

amazon_page = requests.get(amazon_URL, headers=headers)
saturn_page = requests.get(saturn_URL,headers=headers)
mediamarkt_page = requests.get(mediamarkt_URL,headers=headers)

amazon_soup = BeautifulSoup(amazon_page.content, 'html.parser')
saturn_soup = BeautifulSoup(saturn_page.content, 'html.parser')
mediamarkt_soup = BeautifulSoup(mediamarkt_page.content, 'html.parser')


# Extract the titles from each website
amazon_title = amazon_soup.find(id='productTitle').get_text().strip()
saturn_title = saturn_soup.find('h1').get_text().strip()
mediamarkt_title = mediamarkt_soup.find('h1').get_text().strip()

# Print the titles
print("Amazon Title:", amazon_title)
print("Saturn Title:", saturn_title)
print("MediaMarkt Title:", mediamarkt_title)

Amazon Title: Apple iPhone 14 Pro Max (256 GB) - Space Schwarz
Saturn Title: APPLE iPhone 14 Pro Max 256 GB Space Schwarz Dual SIM
MediaMarkt Title: APPLE iPhone 14 Pro Max 256 GB Space Schwarz Dual SIM


In [2]:
# Amazon
amazon_price_whole = amazon_soup.find('span', class_='a-price-whole').get_text().strip()
amazon_price_fraction = amazon_soup.find('span', class_='a-price-fraction').get_text().strip()
amazon_price_symbol = amazon_soup.find('span', class_='a-price-symbol').get_text().strip()
amazon_price_whole = amazon_price_whole.replace(',', '')
amazon_price = f"{amazon_price_whole}.{amazon_price_fraction} {amazon_price_symbol}"


# Saturn
saturn_price_element = saturn_soup.find('span', class_='sc-hLBbgP bxnGMb')
if saturn_price_element is not None:
    saturn_price_whole = saturn_price_element.get_text().strip()
    saturn_price_symbol_element = saturn_price_element.find_previous_sibling('span')
    if saturn_price_symbol_element is not None:
        saturn_price_symbol = saturn_price_symbol_element.get_text().strip()
        saturn_price = f"{saturn_price_symbol} {saturn_price_whole}"
    else:
        saturn_price = saturn_price_whole
else:
    saturn_price = "Price not found"

#Mediamarkt  

mediamarkt_price_element = mediamarkt_soup.find('span', class_='sc-hLBbgP jZttUN')
if mediamarkt_price_element is not None:
    mediamarkt_price = mediamarkt_price_element.get_text().strip()
else:
    mediamarkt_price = "Price not found"






# MediaMarkt
#mediamarkt_price = mediamarkt_soup.find('div', class_='price').get_text().strip()

# Print the title and respective price
print("Amazon Title:", amazon_title)
print("Amazon Price:", amazon_price)
print("Saturn Title:", saturn_title)
print("Saturn Price:", saturn_price)
print("MediaMarkt Title:", mediamarkt_title)
print("MediaMarkt Price:", mediamarkt_price)

Amazon Title: Apple iPhone 14 Pro Max (256 GB) - Space Schwarz
Amazon Price: 1.336.89 €
Saturn Title: APPLE iPhone 14 Pro Max 256 GB Space Schwarz Dual SIM
Saturn Price: € 1439,
MediaMarkt Title: APPLE iPhone 14 Pro Max 256 GB Space Schwarz Dual SIM
MediaMarkt Price: € 1439,


In [3]:
# Define the price threshold for notification so that the user will be notified if the price of the product is reduced below the 
#threshold value set 

price_threshold = 1000

# Convert the prices to floats
amazon_price_float = float(amazon_price.replace('.', '').replace(' €', '').replace(',', '.'))
saturn_price_float = float(saturn_price.replace('.', '').replace(' €', '').replace(',', '').replace('€ ', ''))

# Extract the numerical part of the mediamarkt_price
mediamarkt_price_numerical = mediamarkt_price.split(' ')[1].replace('.', '').replace(',', '.')
mediamarkt_price_float = float(mediamarkt_price_numerical)

# Compare prices and determine the lowest price
lowest_price = min(amazon_price_float, saturn_price_float, mediamarkt_price_float)

# Check if the lowest price is below the threshold
if lowest_price < price_threshold:
    # Send notification to the user (email or push notification)
    notification_message = f"The lowest price ({lowest_price}) is below the threshold ({price_threshold})."
    
    # Send an email
    sender_email = 'your_sender_email@example.com'
    receiver_email = 'your_receiver_email@example.com'
    password = 'your_email_password'
    
    subject = 'Price Notification'
    body = notification_message
    
    message = f'Subject: {subject}\n\n{body}'
    
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
    
    print("Notification sent!")
else:
    print("Price is not below the threshold.")

Price is not below the threshold.


In [ ]:
import csv
import datetime
import time

# Define the filename for the CSV file
csv_filename = 'price_product_data.csv'

# Open the CSV file in append mode
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Run the price checking loop indefinitely
    while True:
        # Get the current timestamp
        current_timestamp = time.time()
        
        # Convert the timestamp to a datetime object
        current_datetime = datetime.datetime.fromtimestamp(current_timestamp)
        
        # Define the prices for different websites 
        amazon_price = 1336.89
        saturn_price = 1439
        mediamarkt_price = 1439
        
        # Write the data to the CSV file
        writer.writerow(['Amazon', 'Amazon Title', amazon_price, current_datetime])
        writer.writerow(['Saturn', 'Saturn Title', saturn_price, current_datetime])
        writer.writerow(['MediaMarkt', 'MediaMarkt Title', mediamarkt_price, current_datetime])
        
        # Define the interval in seconds (24 hours in this example)
        interval = 24 * 60 * 60
        
        # Sleep for the specified interval before checking the prices again
        time.sleep(interval)
